# W3P2 - PART3 - yelp_foursquareEDA

assignment file part 2

In [1]:
from datetime import datetime
import pandas as pd
import requests
import json
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. Assuming the average comfortable walking speed below, the radius of 1km should list sites available within a 15min walking distance from each bike station.

$$
1km * hour/4km 
$$

In [2]:
# Import data from file
df = pd.read_csv('../data/df_ctybks_toronto.csv')
df.shape

(655, 19)

Create lat and long series from the ctyBks table:

In [3]:
lat = df['latitude']
long = df['longitude']
print(f'{lat[5]},{long[5]}')

43.657763,-79.389165


In [4]:
numStns = len(lat)

In [5]:
def fsqStationPOI(lat, long):
  '''
  input:
  output:
  '''
  api_key = os.environ["FOURSQUARE_API_KEY"]
  fields = 'fsq_id,name,geocodes,categories,distance,rating,hours,hours_popular,popularity,features'

  headers = {
      "accept": "application/json",
      "Authorization": api_key
  }

  fsq_url = (f'https://api.foursquare.com/v3/places/search?ll={lat},{long}&fields={fields}&radius=1000&&limit=50')
  
  fsqStnPOI_json = requests.get(fsq_url, headers=headers).json()

  list_of_dict = []
  for poi in fsqStnPOI_json['results']:
    categories = poi.get('categories', None)
    cat_id = categories[0]['id'] if len(categories) > 0 else None
    category_name = categories[0]['name'] if len(categories) > 0 else None

    poi_dict = {
            'lat-long': f'{lat},{long}'
          , 'fsq_id': poi['fsq_id']
          , 'cat_id': cat_id
          , 'category_name': category_name
          , 'categories': categories
          , 'name': poi.get('name', None)
          , 'distance': poi.get('distance', None)
          , 'latitude': poi.get('geocodes', {}).get('main', {}).get('latitude', None)
          , 'longitude': poi.get('geocodes', {}).get('main', {}).get('longitude', None)
          , 'address': poi.get('location', {}).get('formatted_address', None)
          , 'popularity': poi.get('popularity', None)
          , 'open_now': poi.get('hours', {}).get('open_now', None)
          , 'rating': poi.get('rating', None)
          , 'rating_count': poi.get('stats', {}).get('total_ratings', None)
          , 'my_timestamp': datetime.now()
      }
      
    list_of_dict.append(poi_dict)
  return pd.DataFrame(list_of_dict)


In [6]:
# datetime.astimezone(tz=pytz.timezone('America/Toronto'))

In [7]:
# I guess I'll figure out how to convert it later..

# utc = '2022-12-03T18:32:24.600000Z'
# date = datetime.strptime(utc, '%Y-%m-%dT%H:%M:%S.%fZ')
# converted = date.astimezone(tz=pytz.timezone('America/Toronto'))

# print(converted)

In [8]:
print(f'{lat[5]},{long[5]}')

43.657763,-79.389165


In [9]:
# Test
fsqStnPOI_df = fsqStationPOI(lat[5], long[5])
fsqStnPOI_df.head(1)

,lat-long,fsq_id,cat_id,category_name,categories,name,distance,latitude,longitude,address,popularity,open_now,rating,rating_count,my_timestamp
0,"43.657763,-79.389165",537d4d6d498ec171ba22e7fe,13034,Café,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",Jimmy's Coffee,294,43.658525,-79.385436,None,0.972089,False,8.5,None,2022-12-03 19:38:44.660853


Generate a list of DataFrames now for each station site:

In [22]:
fsq_dfs_list = []
for i in range(numStns):
  poi_df = fsqStationPOI(lat[i], long[i])
  fsq_dfs_list.append(poi_df)

Concatenate the DataFrames to create one large one.

In [23]:
df_fsq = x = pd.DataFrame()
x = x = pd.DataFrame()

# concatenate full df object of all stn site data:
for poi_df in fsq_dfs_list:
    x = pd.concat([df_fsq, poi_df])
    df_fsq = x

In [24]:
df_fsq.shape

(32709, 15)

#### SAVE IT!!

In [25]:
df_fsq.to_csv(f'../data/fsqPOI/df_fsq{datetime.now()}.csv', index=False)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [15]:
import requests
import os

def yelpStationPOI(lat, long):
  '''
  input:
  output:
  '''
  api_key = os.environ["YELP_API_KEY"]
  yelp_url = (f'https://api.yelp.com/v3/businesses/search?latitude={lat}&longitude={long}&radius=1000')
  payload={}
  headers = {
      "accept": "application/json",
      "Authorization": f'Bearer {api_key}'
  }

  yelpStnPOI_json = requests.get(yelp_url, headers=headers, data=payload).json()           #Perform get request
  
  list_of_dict = []
  for poi in yelpStnPOI_json['businesses']:
    poi_dict = {
          'lat-long': f'{lat},{long}'
          , 'category_name': poi['categories'][0]['alias']
          , 'categories': poi['categories']
          , 'name': poi['name']
          , 'distance': poi['distance']
          , 'latitude': poi['coordinates']['latitude']
          , 'longitude': poi['coordinates']['longitude']
          , 'address': poi['location']['display_address']
          , 'price': poi.get('price', None)
          , 'is_closed': poi.get('is_closed')
          , 'rating': poi.get('rating')
          , 'rating_count': poi.get('review_count')
          , 'my_timestamp': datetime.now()
      }
      
    list_of_dict.append(poi_dict)
  return pd.DataFrame(list_of_dict)


In [17]:
# Test
yelpStnPOI_dfTest = yelpStationPOI(lat[5], long[5])
yelpStnPOI_dfTest.head(1)

,lat-long,category_name,categories,name,distance,latitude,longitude,address,price,is_closed,rating,rating_count,my_timestamp
0,"43.657763,-79.389165",ramen,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",Sansotei Ramen,376.064159,43.655,-79.38643,"[179 Dundas Street W, Toronto, ON M5G 1Z8, Can...",$$,False,4.0,925,2022-12-03 20:17:17.368564


In [26]:
yelp_dfs_list = []
for i in range(numStns):
  poi_df = yelpStationPOI(lat[i], long[i])
  yelp_dfs_list.append(poi_df)

In [47]:
df_yelp = x = pd.DataFrame()
x = x = pd.DataFrame()

# concatenate full df object of all stn site data:
for poi_df in yelp_dfs_list:
    x = pd.concat([df_yelp, poi_df])
    df_yelp = x

In [48]:
df_yelp.head()

,lat-long,category_name,categories,name,distance,latitude,longitude,address,price,is_closed,rating,rating_count,my_timestamp
0,"43.665269,-79.319796",egyptian,"[{'alias': 'egyptian', 'title': 'Egyptian'}, {...",Maha's,1004.992087,43.671670,-79.328530,"[226 Greenwood Avenue, Toronto, ON M4L 2R2, Ca...",$$,False,4.0,572,2022-12-03 22:09:56.323402
1,"43.665269,-79.319796",burgers,"[{'alias': 'burgers', 'title': 'Burgers'}]",The Burger's Priest,374.238073,43.666700,-79.315585,"[1636 Queen Street E, Toronto, ON M4L 1G3, Can...",$$,False,3.5,498,2022-12-03 22:09:56.323415
2,"43.665269,-79.319796",mexican,"[{'alias': 'mexican', 'title': 'Mexican'}]",Chino Locos Original,467.122546,43.664482,-79.325501,"[4 Greenwood Avenue, Toronto, ON M4L 2P4, Canada]",$,False,4.0,190,2022-12-03 22:09:56.323420
3,"43.665269,-79.319796",halal,"[{'alias': 'halal', 'title': 'Halal'}, {'alias...",Lahore Tikka House,787.473798,43.671480,-79.324510,"[1365 Gerrard Street E, Toronto, ON M4L 1Z3, C...",$$,False,3.5,537,2022-12-03 22:09:56.323424
4,"43.665269,-79.319796",italian,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",Gio Rana's Really Really Nice Restaurant,881.124117,43.663335,-79.330419,"[1220 Queen Street East, Toronto, ON M4M 1L7, ...",$$$,False,4.0,226,2022-12-03 22:09:56.323428


In [49]:
df_yelp.shape

(12827, 13)

### SAVE IT!!!

In [30]:
df_yelp.to_csv(f'../data/yelpPOI/df_yelp{datetime.now()}.csv', index=False)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

## Cleaning and Normalizing the nested table values:

### YELP POI:

The function which created the DataFrame made sure that each POI had at least one category name and alias, but we can see above that many have two or even three which can be normalized to flatten the hierarchy.

In [82]:
# Clone a working copy:
df = df_yelp.copy()

# Normalize the categories:
dfcat = pd.json_normalize(df['categories'][0])
dfcat.head()

,0,1,2
0,"{'alias': 'egyptian', 'title': 'Egyptian'}","{'alias': 'vegan', 'title': 'Vegan'}","{'alias': 'sandwiches', 'title': 'Sandwiches'}"
1,"{'alias': 'italian', 'title': 'Italian'}",None,None
2,"{'alias': 'burgers', 'title': 'Burgers'}",None,None
3,"{'alias': 'newcanadian', 'title': 'Canadian (N...",None,None
4,"{'alias': 'japanese', 'title': 'Japanese'}","{'alias': 'pubs', 'title': 'Pubs'}",None


In [83]:
# normalize the resulting 3 columns into 3 tables in the example below:
dfcat0 = pd.json_normalize(dfcat[0])
dfcat1 = pd.json_normalize(dfcat[1])
dfcat2 = pd.json_normalize(dfcat[2])
dfcat0.head()

,alias,title
0,egyptian,Egyptian
1,italian,Italian
2,burgers,Burgers
3,newcanadian,Canadian (New)
4,japanese,Japanese


In [84]:
# Add new category labels back onto the df: 
df['cat1_alias'] = dfcat0['alias']
df['cat1_title'] = dfcat0['title']

df['cat2_alias'] = dfcat1['alias']
df['cat2_title'] = dfcat1['title']

df['cat3_alias'] = dfcat2['alias']
df['cat3_title'] = dfcat2['title']

# Remove the now redundant 'category' and 'name' columns:
df = df.drop(['categories', 'category_name'], axis = 1)

df.head(1)

,lat-long,name,distance,latitude,longitude,address,price,is_closed,rating,rating_count,my_timestamp,cat1_alias,cat1_title,cat2_alias,cat2_title,cat3_alias,cat3_title
0,"43.665269,-79.319796",Maha's,1004.992087,43.67167,-79.32853,"[226 Greenwood Avenue, Toronto, ON M4L 2R2, Ca...",$$,False,4.0,572,2022-12-03 22:09:56.323402,egyptian,Egyptian,vegan,Vegan,sandwiches,Sandwiches


the addresses look chunky which diminishes the aesthetic of the table, and I expect this format might be useful for printing them in mailing format.  I won't be using it for my analysis at this point anyway, but I would certainly research this more before I made decisions about changing the format.  


In [93]:
# Drop columns from our copy df not currently needed:
df = df.drop(['address', 'cat1_alias', 'cat2_alias', 'cat3_alias'], axis = 1)
df.head(2)

,lat-long,name,distance,latitude,longitude,price,is_closed,rating,rating_count,my_timestamp,cat1_title,cat2_title,cat3_title
0,"43.665269,-79.319796",Maha's,1004.99,43.67167,-79.328530,2.0,False,4.0,572,2022-12-03 22:09:56.323402,Egyptian,Vegan,Sandwiches
1,"43.665269,-79.319796",The Burger's Priest,374.24,43.66670,-79.315585,2.0,False,3.5,498,2022-12-03 22:09:56.323415,Italian,NaN,NaN


flip the "is_closed" column to remove the confusing double negative, and enable foursquare comparison:

In [96]:
df['is_closed'] = df['is_closed'] == False
# Rename the heading to match:
df = df.rename(columns = {'is_closed':'is_open'})
df.head(2)

,lat-long,name,distance,latitude,longitude,price,is_open,rating,rating_count,my_timestamp,cat1_title,cat2_title,cat3_title
0,"43.665269,-79.319796",Maha's,1004.99,43.67167,-79.328530,2.0,True,4.0,572,2022-12-03 22:09:56.323402,Egyptian,Vegan,Sandwiches
1,"43.665269,-79.319796",The Burger's Priest,374.24,43.66670,-79.315585,2.0,True,3.5,498,2022-12-03 22:09:56.323415,Italian,NaN,NaN


Convert the timestamp data to the Toronto's time zone:

In [99]:
from datetime import datetime
import pytz

In [106]:
# create both timezone objects
old_timezone = pytz.timezone("US/Mountain")
new_timezone = pytz.timezone("US/Eastern")

# two-step process
localized_timestamp = old_timezone.localize(testdate)
new_testdate = localized_timestamp.astimezone(new_timezone)
new_testdate

Timestamp('2022-12-04 00:09:56.323402-0500', tz='US/Eastern')

In [109]:
# create both timezone objects
old_timezone = pytz.timezone("US/Mountain")
new_timezone = pytz.timezone("US/Eastern")

# Confirm current timezone
print(df['my_timestamp'].iloc[0])

# two-step process
for i in range(df.shape[0]):
  localized_timestamp = old_timezone.localize(df['my_timestamp'].iloc[i])
  df['my_timestamp'].iloc[i] = localized_timestamp.astimezone(new_timezone)

# Has converted to new timezone:
print(df['my_timestamp'].iloc[0])

/var/folders/sy/_mmt08610_3_rz06wyv14lxh0000gn/T/ipykernel_35521/1236673304.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['my_timestamp'].iloc[i] = localized_timestamp.astimezone(new_timezone)


Timestamp('2022-12-04 00:09:56.323402-0500', tz='US/Eastern')

In [105]:
testdate = df['my_timestamp'].iloc[0]
print(testdate)
# testdate = testdate.timestamp.astimezone(tz=pytz.timezone('America/Toronto'))
# print(testdate)

2022-12-03 22:09:56.323402


In [ ]:
from datetime import datetime
import pytz

utc = '2022-12-03T18:32:24.600000Z'
date = datetime.strptime(utc, '%Y-%m-%dT%H:%M:%S.%fZ')
converted = date.astimezone(tz=pytz.timezone('America/Toronto'))

print(converted)

In [92]:
df['distance'] = round(df['distance'], 2)
df.head(2)

,lat-long,name,distance,latitude,longitude,price,is_closed,rating,rating_count,my_timestamp,cat1_alias,cat1_title,cat2_title,cat3_title
0,"43.665269,-79.319796",Maha's,1004.99,43.67167,-79.328530,2.0,False,4.0,572,2022-12-03 22:09:56.323402,egyptian,Egyptian,Vegan,Sandwiches
1,"43.665269,-79.319796",The Burger's Priest,374.24,43.66670,-79.315585,2.0,False,3.5,498,2022-12-03 22:09:56.323415,italian,Italian,NaN,NaN


##### Categorical encoding: price


In [85]:
print(df['price'].value_counts())
print(df['price'].isna().value_counts())
df['price'].unique()

$$      7356
$       2190
$$$     1210
$$$$     524
Name: price, dtype: int64
False    11280
True      1547
Name: price, dtype: int64


array(['$$', '$', '$$$', None, '$$$$'], dtype=object)

In [86]:
# order in array above will be same as the keys,
keys = df['price'].unique()
# Setting ordinal numerical values to match the order:
values = [2, 1, 3, None, 4]
price_map = dict(zip(keys, values))
price_map

{'$$': 2, '$': 1, '$$$': 3, None: None, '$$$$': 4}

In [87]:
# Replace
df['price'] = df['price'].map(price_map)
# Validate counts unaffected:
print(df['price'].value_counts())

df.head(1)

2.0    7356
1.0    2190
3.0    1210
4.0     524
Name: price, dtype: int64


,lat-long,name,distance,latitude,longitude,address,price,is_closed,rating,rating_count,my_timestamp,cat1_alias,cat1_title,cat2_alias,cat2_title,cat3_alias,cat3_title
0,"43.665269,-79.319796",Maha's,1004.992087,43.671670,-79.328530,"[226 Greenwood Avenue, Toronto, ON M4L 2R2, Ca...",2.0,False,4.0,572,2022-12-03 22:09:56.323402,egyptian,Egyptian,vegan,Vegan,sandwiches,Sandwiches
1,"43.665269,-79.319796",The Burger's Priest,374.238073,43.666700,-79.315585,"[1636 Queen Street E, Toronto, ON M4L 1G3, Can...",2.0,False,3.5,498,2022-12-03 22:09:56.323415,italian,Italian,NaN,NaN,NaN,NaN
2,"43.665269,-79.319796",Chino Locos Original,467.122546,43.664482,-79.325501,"[4 Greenwood Avenue, Toronto, ON M4L 2P4, Canada]",1.0,False,4.0,190,2022-12-03 22:09:56.323420,burgers,Burgers,NaN,NaN,NaN,NaN
3,"43.665269,-79.319796",Lahore Tikka House,787.473798,43.671480,-79.324510,"[1365 Gerrard Street E, Toronto, ON M4L 1Z3, C...",2.0,False,3.5,537,2022-12-03 22:09:56.323424,newcanadian,Canadian (New),NaN,NaN,NaN,NaN
4,"43.665269,-79.319796",Gio Rana's Really Really Nice Restaurant,881.124117,43.663335,-79.330419,"[1220 Queen Street East, Toronto, ON M4M 1L7, ...",3.0,False,4.0,226,2022-12-03 22:09:56.323428,japanese,Japanese,pubs,Pubs,NaN,NaN


In [88]:
df['cat1_alias'].value_counts()

japanese            1936
italian             1282
egyptian            1278
breakfast_brunch    1277
burgers              653
newcanadian          653
ramen                652
lounges              647
sandwiches           646
seafood              643
tapasmallplates      641
thai                 638
vietnamese           631
icecream             626
pizza                624
Name: cat1_alias, dtype: int64

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating